In [3]:
import os
import sys
from pathlib import Path
base_dir = str(Path(Path.cwd()).resolve().parent.parent)
print(base_dir)
sys.path.append(base_dir)
import json
import pandas as pd
from src.retriever import Retriever, load_retriever

B:\data_science_projects\erdos_institute\aware-nlp


## Load retriever to merge reddit_ids with statements

In [5]:
ret_model_name = 'multi-qa-mpnet-base-dot-v1_50-overlap'
attribute_filename = f'../../data/best_buy/{ret_model_name}.pkl'
embedding_filename = f'../../data/best_buy/{ret_model_name}_embeddings.npy'
ret = load_retriever(attribute_filename, embedding_filename)

## Create composite labels from manually labelled data

In [37]:
import pandas as pd
import numpy as np

published_urls = {
    'peter': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSpSp39dL7fapMm2wZD_AQriiXFB81j1M7UyZJPuGtsE1p0fpiWKX-cyYDy10SceVJol1kc-CrCe-ZZ/pub?gid=1213933157&single=true&output=csv',
    'craig': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRdU37NHc1rufQXqgFj8x3yVFLNHwsLuDXHN_-LHduZWQEc4uq0T7q8mleVBVclZPmdcV36EdX9xIIY/pub?output=csv',
    'anil': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQSq5DnR6MoQ-W6L0fuliZfeql5SRrjeCtgf0cOwZRnltTM7NMiYv-8Lxsj169iqNSs-p5lXzt_1tW1/pub?output=csv',
    'himanshu': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQnpgHyoYuuzdD-CVcFWosqJ7EGUITAhE7XW-X1-mzAfsbhqg_nItuP9vt9OOUq1ACXhFaZfzsiaagP/pub?output=csv',
    'mohammad': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSLne6nuIwlLUy_c9pMRuE27dnD68YKY6wftO-0CYN_7WnP7iqUX2VWY5qC4fuSyNK9JVDZOTp16WhD/pub?output=csv',
    'baian': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTeDJ8M07sVMHCBhAXk3lHe3V-ZTL-G5CF3yPdST9Fc8Vp9yhAqw-KSjkBNkqnlsKmQTkxrWSFzOqG5/pub?output=csv'
}
labellers = list(published_urls.keys()) + ['observer']
labelled_sample_dfs = [pd.read_csv(url) for url in published_urls.values()] 
df = labelled_sample_dfs[0]
for i in range(1, len(labelled_sample_dfs)):
    df = df.merge(labelled_sample_dfs[i], how='outer', on=['question', 'statement'])
df.rename(columns = {name: name.lower() for name in df.columns}, inplace=True)
df['average_label'] = df[labellers].mean(axis=1)
df['label_sum'] = df[labellers].sum(axis=1)
df['consensus'] = (df['label_sum']>=2).astype(int)

In [38]:
questions = df.question.unique()
question = questions[1]
df[df.question==question].sort_values('label_sum', ascending=False).reset_index(drop=True)
questions = df.question.unique()
resampled_df = pd.DataFrame()
for question in questions:
    temp_df = df[df.question==question].sort_values('label_sum', ascending=False).reset_index(drop=True)
    idxs = np.append(np.arange(0,10), np.arange(len(temp_df) - 20, len(temp_df)))
    resampled_df = pd.concat([resampled_df, temp_df.iloc[idxs,:]])
resampled_df.to_csv('../../data/best_buy/evaluation_set.csv', index=False)

In [40]:
questions = df.question.unique()
resampled_df = pd.DataFrame()
for question in questions:
    temp_df = df[df.question==question].sort_values('label_sum', ascending=False).reset_index(drop=True)
    idxs = np.append(np.arange(0,10), np.arange(len(temp_df) - 20, len(temp_df)))
    resampled_df = pd.concat([resampled_df, temp_df.iloc[idxs,:]])
resampled_df.to_csv('../../data/best_buy/evaluation_set.csv', index=False)

In [41]:
labelled_dataset = pd.read_csv('../../data/best_buy/evaluation_set.csv')

In [42]:
chunk_df = pd.DataFrame({'statement':ret.chunks, 'reddit_id': ret.chunk_ids})
chunk_df

,statement,reddit_id
0,Research Study Recruitment - Managers,12opsul
1,![gif](giphy|YmQLj2KxaNz58g7Ofg) $50?,jgjgy9e
2,bullshit on getting $50 its just a coupon to B...,jgjpqmp
3,Sr role eliminated; Good luck y’all,12m0ozl
4,I was there almost 5 years before my position ...,jg9its6
...,...,...
5659,You do realize you’re asking for a bonus while...,kttl60a
5660,HERE YOUR BONUS🥒,kttmfik
5661,What has leadership done to deserve a bonus?,kttj3mt
5662,Board meeting finished this week. It’s always...,ktt37u3


In [43]:
with open('../../data/BestBuyWorkers.json', 'r') as f:
    workers = json.load(f)
worker_df = pd.DataFrame(workers)

In [57]:
labelled_dataset

,question,statement,peter,craig,anil,himanshu,observer,mohammad,baian,average_label,label_sum,consensus
0,What do Best Buy employees think fo the company?,Move on and look for something better. Or hang...,1,1,1,1,1,1.0,1,1.0,7.0,1
1,What do Best Buy employees think fo the company?,"Do yourself a favor, don't. It's not worth it...",1,1,1,1,1,1.0,1,1.0,7.0,1
2,What do Best Buy employees think fo the company?,"Don’t listen to this guy, I work there and the...",1,1,1,1,1,1.0,1,1.0,7.0,1
3,What do Best Buy employees think fo the company?,My bestbuy doesn’t care about the cashiers at ...,1,1,1,1,1,1.0,1,1.0,7.0,1
4,What do Best Buy employees think fo the company?,Best Buy used to be a great company to work fo...,1,1,1,1,1,1.0,1,1.0,7.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
85,Do employees feel understaffed?,Sounds like when we had AP and people coming i...,0,0,0,0,0,0.0,0,0.0,0.0,0
86,Do employees feel understaffed?,How many was in stock?; I'm in hawaii. It's 6:...,0,0,0,0,0,0.0,0,0.0,0.0,0
87,Do employees feel understaffed?,"With the upmost respect, the employees are rea...",0,0,0,0,0,0.0,0,0.0,0.0,0
88,Do employees feel understaffed?,"Yea, never the employees tho",0,0,0,0,0,0.0,0,0.0,0.0,0


In [68]:
temp_df = labelled_dataset[labelled_cols].merge(
        chunk_df, on='statement', how='left'
    )
temp_df.groupby(['statement','question']).first().reset_index()

,statement,question,label_sum,average_label,consensus,reddit_id
0,![gif](giphy|znRstrOYuirrW),What do Best Buy employees think fo the company?,0.0,0.000000,0,ju59a0l
1,#shockedpikachuface,What do Best Buy employees think fo the company?,0.0,0.000000,0,hhgbgps
2,12 hr shift here too. Normal pay man,What are the most common reasons for employees...,2.0,0.285714,1,iwshpu5
3,12-day Application Review; What is the usual d...,What are the most common reasons for employees...,0.0,0.000000,0,klk0z1
4,Absolutely. I had a talk with a leader last we...,Do employees feel understaffed?,7.0,1.000000,1,ibg921p
...,...,...,...,...,...,...
85,done company wide every March not on your work...,What do Best Buy employees think fo the company?,0.0,0.000000,0,k01yx5z
86,i take only cash tips from anyone thats not a ...,Do employees feel understaffed?,0.0,0.000000,0,ki7m0f4
87,"i've been here for 8 years, work in the highes...",What are the most common reasons for employees...,7.0,1.000000,1,jag8nz0
88,nah,What do Best Buy employees think fo the company?,0.0,0.000000,0,ixd7txp


In [64]:
val_counts = temp_df.statement.value_counts()
val_counts[val_counts.values>1]

statement
[deleted]                                31
[removed]                                 9
Thank you so much!                        3
Yeah thats the main way to learn shir     2
Name: count, dtype: int64

In [74]:
labeled_cols = ['question', 'label_sum', 'average_label', 'consensus', 'statement']
worker_cols = ['aware_post_type', 'aware_created_ts', 'reddit_id', 'reddit_link_id','reddit_parent_id', 'reddit_permalink', 'reddit_subreddit']
temp_df = labelled_dataset[labelled_cols].merge(
        chunk_df, on='statement', how='left'
    ).groupby(['statement','question']).first().reset_index()
dataset_df = temp_df.merge(
        worker_df[worker_cols], on='reddit_id', how='left'
    )
dataset_df = dataset_df.replace({'What do Best Buy employees think fo the company?':'What do Best Buy employees think of the company?'})

In [80]:
dataset_df.to_csv('../../data/best_buy/questions_statements_labels.csv')

In [45]:
from langchain_community.document_loaders import DataFrameLoader

In [77]:
from datasets import Dataset

In [78]:
dset = Dataset.from_pandas(dataset_df)
dset

Dataset({
    features: ['statement', 'question', 'label_sum', 'average_label', 'consensus', 'reddit_id', 'aware_post_type', 'aware_created_ts', 'reddit_link_id', 'reddit_parent_id', 'reddit_permalink', 'reddit_subreddit'],
    num_rows: 90
})

In [75]:
from langchain_community.document_loaders import DataFrameLoader
def parse_dataset(dataset_df: pd.DataFrame, question: str):
    df = dataset_df[dataset_df.question==question]
    loader = DataFrameLoader(df, page_content_column='statement')
    return loader.load()

questions = dataset_df.question.unique()
dataset_dict = {f'question_{i}':parse_dataset(dataset_df, q) for i, q in enumerate(questions)}

In [76]:
dataset_dict['question_0']

[Document(page_content='![gif](giphy|znRstrOYuirrW)', metadata={'question': 'What do Best Buy employees think of the company?', 'label_sum': 0.0, 'average_label': 0.0, 'consensus': 0, 'reddit_id': 'ju59a0l', 'aware_post_type': 'comment', 'aware_created_ts': '2023-07-30T21:11:35', 'reddit_link_id': 't3_15e1vvl', 'reddit_parent_id': 't3_15e1vvl', 'reddit_permalink': '/r/BestBuyWorkers/comments/15e1vvl/customer_posts_are_disallowed_on_this_subreddit/ju59a0l/', 'reddit_subreddit': 'BestBuyWorkers'}),
 Document(page_content='#shockedpikachuface', metadata={'question': 'What do Best Buy employees think of the company?', 'label_sum': 0.0, 'average_label': 0.0, 'consensus': 0, 'reddit_id': 'hhgbgps', 'aware_post_type': 'comment', 'aware_created_ts': '2021-10-21T00:06:32', 'reddit_link_id': 't3_qafhhx', 'reddit_parent_id': 't1_hh3zf24', 'reddit_permalink': '/r/BestBuyWorkers/comments/qafhhx/we_can_improve_our_working_conditions_by_working/hhgbgps/', 'reddit_subreddit': 'BestBuyWorkers'}),
 Docu